### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [ ]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [ ]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [ ]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding="utf-8")

Explicit данные

In [ ]:
ratings.head(10)

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [ ]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [ ]:
implicit_ratings.head(10)

Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() for x in model.similar_items(item_id)]

In [ ]:
model.similar_items(1)

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [ ]:
from factorization import SVD, SVDS
from datetime import datetime

In [ ]:
get_similars = lambda item_id, model : movie_info[movie_info["movie_id"].isin(model.similar_items(item_id))][["name", "category"]]

get_recommendations = lambda user_id, model : movie_info[movie_info["movie_id"].isin(model.recommend(user_id))][["name", "category"]]

get_user_history = lambda user_id, implicit_ratings : movie_info[movie_info["movie_id"].isin(implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"])][["name", "category"]]

In [ ]:
def test(model):
    item_id = 1
    target = movie_info[movie_info.movie_id == item_id]
    print(f"Similar to {target.name.item()} ({target.category.item()}):")
    sims = model.similar_items(item_id)
    
    for idx, i in enumerate(sims):
        selected = movie_info[movie_info.movie_id == i]
        print(f"{idx + 1}) {selected.name.item()} ({selected.category.item()})")
    
    print()
    
    user_id = 4
    print(f"Recommended movies for user {user_id}:")
    recs = model.recommend(user_id)
    
    for idx, i in enumerate(recs):
        selected = movie_info[movie_info.movie_id == i]
        if len(selected) != 0:
            print(f"{idx + 1}) {selected.name.item()} ({selected.category.item()})")

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [ ]:
user_item_exp = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

In [ ]:
%%time
model_svd = SVD(64, iterations=1000, lr=1e-4, verbose=True, weight_decay=1e-2)
model_svd.fit(user_item_exp)  

timestamp = datetime.now()
model_svd.logger.save(f"log.{timestamp.date()}_{timestamp.time()}.csv")
model_svd.logger.plot("iter", "rmse", label="RMSE", x_label="Iteration", y_label="RMSE")

In [ ]:
# test(model_svd)

In [ ]:
%%time
model_svds = SVDS(64, iterations=50, lr=1e-2, verbose=True, weight_decay=1e-2, save_every=10)

model_svds.fit(user_item_exp)

timestamp = datetime.now()
model_svds.logger.save(f"log.{timestamp.date()}_{timestamp.time()}.csv")
model_svds.logger.plot("iter", "rmse", label="RMSE", x_label="Iteration", y_label="RMSE")

In [ ]:
# test(model_svds)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
from factorization import ALS

In [ ]:
%%time
model_als = ALS(64, 10, 1e-5, 1e-5, True)

model_als.fit(user_item_csr)

timestamp = datetime.now()
model_als.logger.save(f"log.{timestamp.date()}_{timestamp.time()}.csv")
model_als.logger.plot("iter", "rmse", label="RMSE", x_label="Iteration", y_label="RMSE")

In [ ]:
# test(model_als)

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [ ]:
from factorization import BPR

In [ ]:
%%time
model_bpr = BPR(64, int(1e4), 1e-1, 1e-2, True, save_every=500)

model_bpr.fit(user_item_csr, batch_size=3000)

timestamp = datetime.now()
model_bpr.logger.save(f"log.{timestamp.date()}_{timestamp.time()}.csv")
model_bpr.logger.plot("iter", "AUC", label="AUC", x_label="Iteration", y_label="AUC")
model_bpr.logger.plot("iter", "AUC2", label="AUC2", x_label="Iteration", y_label="AUC2")

In [ ]:
# test(model_bpr)

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [ ]:
test(model_svd)

In [ ]:
test(model_svds)

In [ ]:
test(model_als)

In [ ]:
test(model_bpr)